In [1]:
from tqdm import tqdm
import os
import time
import pandas as pd
import json
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration

In [2]:
from PIL import Image, ImageDraw, ImageFont

# Function to add a tilted watermark to an image
def add_watermark_to_image(image_path, watermark_text, font_path='arial.ttf', angle=-45):
    # Open the original image
    original = Image.open(image_path)

    # Make the image editable by converting it to RGBA
    original = original.convert("RGBA")

    # Set up the font for the watermark text
    fnt_size = min(original.size) // 10  # Dynamic font size based on image size
    fnt = ImageFont.truetype(font_path, fnt_size)

    # Create an image for the text to get the correct size
    text_img = Image.new('RGBA', original.size, (255, 255, 255, 0))
    d = ImageDraw.Draw(text_img)
    
    # Get text size
    textwidth, textheight = d.textsize(watermark_text, font=fnt)
    
    # Create a new image with a size to fit the text and rotate it
    text_img = Image.new('RGBA', (textwidth, textheight), (255, 255, 255, 0))
    d = ImageDraw.Draw(text_img)
    d.text((0, 0), watermark_text, font=fnt, fill=(255, 255, 255, 128))

    # Rotate the text image
    rotated_text_img = text_img.rotate(angle, expand=1)

    # Calculate position for the rotated text
    width, height = original.size
    text_width, text_height = rotated_text_img.size
    x = (width - text_width) // 2
    y = (height - text_height) // 2

    # Paste the rotated text image onto the original image
    original.paste(rotated_text_img, (x, y), rotated_text_img)

    # Convert to RGB and save the image with watermark
    watermarked = original.convert("RGB")
    watermarked.save("watermarked_image.jpg")

# Example usage
image_path = 'img.png'  # Replace with the path to your image
watermark_text = 'WATERMARK'  # Replace with your watermark text
font_path = 'arial.ttf'  # Replace with the path to your .ttf font file

# Add watermark to the image
add_watermark_to_image(image_path, watermark_text, font_path)


/tmp/ipykernel_14525/1039404886.py:20: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  textwidth, textheight = d.textsize(watermark_text, font=fnt)


In [3]:
from function.extract_caption import extract_caption

In [4]:
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-flan-t5-xl")
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-flan-t5-xl")
device = "cuda:3"
model.to(device)

caption = extract_caption("watermarked_image.jpg", model, processor, device)
print(caption)


The image features a small bird standing on the surface of a body of water, with its wings spread out. The bird appears to be perched on the edge of the water, ready to fly away as it awaits its prey. There are several other birds present in the scene, including a red-winged blackbird near the left side of the image and a blue jay near the right side of the image. All of these birds seem to be enjoying their time in the water, which is a common sight in many parts of the world.
